<a href="https://colab.research.google.com/github/zaki-moh/MMA_fight_predictor/blob/main/mma_predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files
uploaded = files.upload()

Saving large_dataset.csv to large_dataset.csv
Saving ufc-fighters-statistics.csv to ufc-fighters-statistics.csv


In [3]:
import pandas as pd
stats = pd.read_csv("ufc-fighters-statistics.csv")
historic_fights = pd.read_csv("large_dataset.csv")
# stats.head()
# stats.isnull().sum()
# print(stats.head(15))
# print(historic_fights.head(10))
#print(stats.columns)
stats.columns.tolist()

#

['name',
 'nickname',
 'wins',
 'losses',
 'draws',
 'height_cm',
 'weight_in_kg',
 'reach_in_cm',
 'stance',
 'date_of_birth',
 'significant_strikes_landed_per_minute',
 'significant_striking_accuracy',
 'significant_strikes_absorbed_per_minute',
 'significant_strike_defence',
 'average_takedowns_landed_per_15_minutes',
 'takedown_accuracy',
 'takedown_defense',
 'average_submissions_attempted_per_15_minutes']

In [4]:
# stats.info()
# stats.describe()
# stats.shape
# print(stats.columns.tolist())

In [5]:
historic_fights = historic_fights.iloc[50:]
#print(historic_fights)
historic_fights = historic_fights[['r_fighter', 'b_fighter', 'winner']].copy()
historic_fights['winner'] = historic_fights.apply(
    lambda row: row['r_fighter'] if row['winner'].lower() == 'red' else row['b_fighter'],
    axis = 1
)

stats['height_cm'] = stats['height_cm'].fillna(stats['height_cm'].median())
stats['date_of_birth'] = pd.to_datetime(stats['date_of_birth'], errors='coerce')
today = pd.to_datetime('today')
stats['age'] = (today - stats['date_of_birth']).dt.days // 365
stats['age'] = stats['age'].fillna(stats['age'].median())
mean_strikes = stats['significant_strikes_landed_per_minute'].mean()
stats['normalized_strikes'] = stats['significant_strikes_landed_per_minute'] / mean_strikes
stats['performance'] = (0.3 * stats['normalized_strikes']
                        + 0.3 * stats['significant_strike_defence'] / 100
                        + 0.2 * stats['takedown_accuracy'] / 100
                        + 0.2 * stats['takedown_defense'] / 100)
stats['strike_efficiency'] = (
    stats['significant_strikes_landed_per_minute']
    - stats['significant_strikes_absorbed_per_minute']
)
stats['grapple_efficiency'] = (
    0.5 * stats['takedown_accuracy'] / 100 + 0.5 * stats['takedown_defense'] / 100
)
stats['win_indicator'] = (stats['wins'] > stats['losses']).astype(int)
stats['win_ratio'] = stats['wins'] / (stats['wins'] + stats["losses"] + stats['draws'])
stats['win_ratio'] = stats['win_ratio'].fillna(0)

merged = historic_fights.merge(
    stats.add_prefix('r_'),
    left_on='r_fighter',
    right_on='r_name'
)
merged = merged.merge(
    stats.add_prefix('b_'),
    left_on='b_fighter',
    right_on='b_name'
)

merged['height_diff'] = merged['r_height_cm'] - merged['b_height_cm']
merged['reach_diff'] = merged['r_reach_in_cm'] - merged['b_reach_in_cm']
merged['age_diff'] = merged['r_age'] - merged['b_age']
merged['strike_eff_diff'] = merged['r_strike_efficiency'] - merged['b_strike_efficiency']
merged['grapple_eff_diff'] = merged['r_grapple_efficiency'] - merged['b_grapple_efficiency']
merged['performance_diff'] = merged['r_performance'] - merged['b_performance']
merged['win_ratio_diff'] = merged['r_win_ratio'] - merged['b_win_ratio']
merged['r_fighter_win'] = (merged['winner'] == merged['r_fighter']).astype(int)
merged = merged[['r_fighter', 'b_fighter', 'height_diff', 'reach_diff', 'age_diff', 'strike_eff_diff', 'grapple_eff_diff', 'performance_diff', 'win_ratio_diff', 'r_fighter_win']].copy()
#print(merged.columns.tolist())
#merged.head()

In [6]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

numeric_cols = ['height_diff',	'reach_diff',	'age_diff',	'strike_eff_diff',	'grapple_eff_diff',	'performance_diff',	'win_ratio_diff']

merged[numeric_cols] = scaler.fit_transform(merged[numeric_cols])

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

X = merged[['height_diff',	'reach_diff',	'age_diff',	'strike_eff_diff',	'grapple_eff_diff',	'performance_diff',	'win_ratio_diff']]
y = merged['r_fighter_win']
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

model = RandomForestClassifier(n_estimators=200, class_weight='balanced', random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.704004329004329
              precision    recall  f1-score   support

           0       0.59      0.44      0.50       636
           1       0.74      0.84      0.79      1212

    accuracy                           0.70      1848
   macro avg       0.67      0.64      0.65      1848
weighted avg       0.69      0.70      0.69      1848



In [8]:
def predictWinner(fighter_A_Name, fighter_B_Name):
    # Lookup both fighters
    fighter_A = stats.loc[stats['name'] == fighter_A_Name].squeeze()
    fighter_B = stats.loc[stats['name'] == fighter_B_Name].squeeze()

    if fighter_A.empty or fighter_B.empty:
        print("One or both fighters not found in the dataset.")
        return

    input_data = pd.DataFrame({
        'height_diff': [fighter_A['height_cm'] - fighter_B['height_cm']],
        'reach_diff': [fighter_A['reach_in_cm'] - fighter_B['reach_in_cm']],
        'age_diff': [fighter_A['age'] - fighter_B['age']],
        'strike_eff_diff': [fighter_A['strike_efficiency'] - fighter_B['strike_efficiency']],
        'grapple_eff_diff': [fighter_A['grapple_efficiency'] - fighter_B['grapple_efficiency']],
        'performance_diff': [fighter_A['performance'] - fighter_B['performance']],
        'win_ratio_diff': [fighter_A['win_ratio'] - fighter_B['win_ratio']]
    })

    input_data[numeric_cols] = scaler.transform(input_data[numeric_cols])

    probs = model.predict_proba(input_data)[0]
    r_win_prob = probs[1]
    b_win_prob = probs[0]

    print(f"\n{fighter_A_Name} 🆚 {fighter_B_Name}")
    print(f"{fighter_A_Name} win probability: {r_win_prob:.2%}")
    print(f"{fighter_B_Name} win probability: {b_win_prob:.2%}")

    winner = fighter_A_Name if r_win_prob > b_win_prob else fighter_B_Name
    print(f"🏆 Predicted Winner: {winner}")

    return winner


In [18]:
predictWinner("Arman Tsarukyan", "Dan Hooker")


Arman Tsarukyan 🆚 Dan Hooker
Arman Tsarukyan win probability: 89.00%
Dan Hooker win probability: 11.00%
🏆 Predicted Winner: Arman Tsarukyan


'Arman Tsarukyan'